# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（最終結果に `ORDER BY` は付けない）
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は **ID 基準**、表示は仕様どおり

## 1) 問題

* `Orders` から **最も多く注文した顧客の `customer_number`** を返す
  *テスト条件: 最大は一意。Follow up: 最大同率が複数でも全件返す。*

* 入力:

  ```text
  Table: Orders
  +-----------------+----------+
  | Column Name     | Type     |
  +-----------------+----------+
  | order_number    | int      | -- PK
  | customer_number | int      |
  +-----------------+----------+
  ```

* 出力:

  ```text
  +-----------------+
  | customer_number |
  +-----------------+
  ```

## 2) 最適解（単一クエリ）

> **集計 → ウィンドウ順位付け**。同率最大にも対応（Follow up 充足）。

```sql
WITH cnt AS (
  SELECT
    customer_number,
    COUNT(*) AS order_cnt
  FROM Orders
  GROUP BY customer_number
),
win AS (
  SELECT
    customer_number,
    DENSE_RANK() OVER (ORDER BY order_cnt DESC) AS rnk
  FROM cnt
)
SELECT
  customer_number
FROM win
WHERE rnk = 1;

Runtime 229 ms
Beats 89.86%

```

### 代替（最大が一意の前提で最短）

> 一意最大だけで良いなら、最小コストで済むことが多いです。

```sql
SELECT customer_number
FROM Orders
GROUP BY customer_number
ORDER BY COUNT(*) DESC
LIMIT 1;

Runtime 230 ms
Beats 88.20%

```

### 代替（Follow up：最大同率すべて）

> ウィンドウを使わず、**最大値＝サブクエリ**で一致抽出。

```sql
SELECT customer_number
FROM Orders
GROUP BY customer_number
HAVING COUNT(*) = (
  SELECT COUNT(*) AS mx
  FROM Orders
  GROUP BY customer_number
  ORDER BY mx DESC
  LIMIT 1
);

Runtime 232 ms
Beats 84.30%

```

### （参考）LATERAL で「上位 k（ここでは 1）」を直接引く

```sql
SELECT s.customer_number
FROM LATERAL (
  SELECT customer_number
  FROM Orders
  GROUP BY customer_number
  ORDER BY COUNT(*) DESC
  LIMIT 1
) AS s;

Runtime 231 ms
Beats 86.09%

```

## 3) 要点解説

* **ウィンドウ関数**: `DENSE_RANK() OVER (ORDER BY COUNT(*) DESC)` を使えば、同数最大も自然に拾える。
  集計結果にだけ順位付けするため、まず `GROUP BY` でデータを縮小してから適用。
* **代替の `HAVING = (SELECT ... LIMIT 1)`**: ウィンドウ不要で読みやすく、結合も発生しないため軽い計画になりやすいです。
* **インデックス推奨**: 集計キーに B-tree

  ```sql
  CREATE INDEX IF NOT EXISTS idx_orders_customer ON Orders (customer_number);
  ```

  `GROUP BY customer_number` のハッシュ集計/ソートが効率化されます。
* **NULL 取扱い**: 仕様上 `customer_number` が必須なら問題なし。NULL 行が混入の可能性がある場合は
  `WHERE customer_number IS NOT NULL` を前置きして堅牢化。

## 4) 計算量（概算）

* `GROUP BY`（顧客数を M、全行数を N とすると）: **O(N)**～**O(N log N)**
  （ハッシュ集計なら近似 O(N)）
* ウィンドウ `DENSE_RANK` は *集計後* の M 行に対して **O(M log M)**（内部ソート）。
  代替の `HAVING = (SELECT ... LIMIT 1)` は **O(M)** 近似。

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Orders] --> B["customer_number ごとに COUNT 集計"]
  B --> C["ウィンドウ DENSE_RANK で順位付け"]
  C --> D[rnk=1 を抽出]
  D --> E[出力 customer_number]
```
